In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torchvision, torch

In [4]:
import sys, os
sys.path.insert(0,"../torchxrayvision/")
import torchxrayvision as xrv
import matplotlib.pyplot as plt
import torch
from torch.nn import functional as F
import glob
import numpy as np
import skimage, skimage.filters
%matplotlib inline
import captum, captum.attr
import torch, torch.nn
import pickle
import attribution
import pandas as pd
import datasets
import shutil
import pickle


In [9]:
ae = xrv.autoencoders.ResNetAE(weights="101-elastic").cuda()

In [10]:
models = [xrv.models.DenseNet(weights="all").cuda(),
          xrv.models.DenseNet(weights="mimic_ch").cuda(),
          xrv.baseline_models.jfhealthcare.DenseNet().cuda()]

In [25]:
for_eval = {
            "Cardiomegaly":datasets.get_data("nih"),
            'Mass':datasets.get_data("nih"),
            #'Nodule':datasets.get_data("nih"), 
            "Atelectasis":datasets.get_data("nih"),
            "Effusion":datasets.get_data("nih"),
            "Lung Opacity":datasets.get_data("rsna"),
            "Pneumothorax":datasets.get_data("siim"),
            }

NIH_Dataset num_samples=67310 views=['PA'] data_aug=None
NIH_Dataset num_samples=67310 views=['PA'] data_aug=None
NIH_Dataset num_samples=67310 views=['PA'] data_aug=None
NIH_Dataset num_samples=67310 views=['PA'] data_aug=None
RSNA_Pneumonia_Dataset num_samples=14511 views=['PA'] data_aug=None
SIIM_Pneumothorax_Dataset num_samples=12954 data_aug=None


In [1]:
# datasets.get_data("vin").pathologies

In [16]:
# for_eval = {
#             #"Cardiomegaly":datasets.get_data("vin"),
#             "Atelectasis":datasets.get_data("vin"),
#             "Effusion":datasets.get_data("vin"),
#             "Lung Opacity":datasets.get_data("vin"),
#             "Pneumothorax":datasets.get_data("vin"),
#             }

VinBrain_Dataset num_samples=15000 views=None data_aug=None
VinBrain_Dataset num_samples=15000 views=None data_aug=None
VinBrain_Dataset num_samples=15000 views=None data_aug=None
VinBrain_Dataset num_samples=15000 views=None data_aug=None


In [ ]:
perf = pd.DataFrame()
for ttarget in for_eval:
    for tmodel in models:
        print(ttarget,tmodel)
        if ttarget in tmodel.pathologies:
            dataset = for_eval[ttarget]
            limit = 32
            auc = attribution.test_epoch(tmodel, for_eval[ttarget], ttarget, limit=limit, batch_size=1)
            h = {}
            h["model"] = str(tmodel)
            h["target"] = ttarget
            h["auc"] = auc
            h["limit"] = limit
            
            perf = perf.append(h, ignore_index=True)
    

In [175]:
#pickle.dump(perf, open("perf.pkl", "bw"))

In [176]:
perf = pickle.load(open("perf.pkl", "br"))

In [194]:
perf.groupby(["target", "model"])[["auc"]].mean().unstack(level=1)

auc                           \
model        XRV-DenseNet121-all XRV-DenseNet121-mimic_ch   
target                                                      
Atelectasis             0.782547                 0.706442   
Cardiomegaly            0.907928                 0.697739   
Effusion                0.870970                 0.800678   
Lung Opacity            0.845384                 0.757021   
Mass                    0.828224                      NaN   
Pneumothorax            0.785364                 0.679494   

                                       
model        jfhealthcare-DenseNet121  
target                                 
Atelectasis                  0.778893  
Cardiomegaly                 0.901575  
Effusion                     0.872704  
Lung Opacity                      NaN  
Mass                              NaN  
Pneumothorax                      NaN

In [ ]:
k = pd.DataFrame()
for ttarget in for_eval:
    for tmodel in models:
        print(ttarget,tmodel)
        if ttarget in tmodel.pathologies:
            dataset = for_eval[ttarget]
            a = attribution.run_eval(ttarget, dataset, tmodel, ae=ae, limit=80)
            a["model"] = str(tmodel)
            k = k.append(a)
            #print(a)
    

In [180]:
import pickle
pickle.dump(k, open("ious80.pkl", "bw"))

In [181]:
k = pickle.load(open("ious80.pkl", "br"))

In [17]:
del k["idx"]

In [182]:
k.head()

,iou,precision,recall,idx,p,target,method,model
0,0.276051,0.432665,0.432665,980,0.786204,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
1,0.352275,0.521011,0.521011,1686,0.882725,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
2,0.424728,0.596223,0.596223,1918,0.708709,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
3,0.265393,0.419463,0.419463,2121,0.509952,Cardiomegaly,latentshift-max,XRV-DenseNet121-all
4,0.367777,0.537773,0.537773,3356,0.626276,Cardiomegaly,latentshift-max,XRV-DenseNet121-all


In [187]:
kk = k.groupby(["target", "model","method"]).agg({'iou' : ['mean', 'std']}).round(2)
#kk

In [189]:
for c in kk.columns.get_level_values(0).unique():
    kk[c+"_"] =  kk[(c, "mean")].apply("{0:0.2f}".format) + "$\pm$" + kk[(c, "std")].apply("{0:0.2f}".format)
    del kk[c]
    kk[c] = kk[c+"_"]
    del kk[c+"_"]

In [190]:
kk.unstack(level=1)

iou                           \
                                                                            
model                        XRV-DenseNet121-all XRV-DenseNet121-mimic_ch   
target       method                                                         
Atelectasis  grad                  0.07$\pm$0.07            0.06$\pm$0.07   
             guided                0.09$\pm$0.08            0.04$\pm$0.04   
             integrated            0.05$\pm$0.05            0.04$\pm$0.05   
             latentshift-max       0.11$\pm$0.12            0.08$\pm$0.11   
Cardiomegaly grad                  0.35$\pm$0.05            0.25$\pm$0.09   
             guided                0.28$\pm$0.06            0.15$\pm$0.06   
             integrated            0.27$\pm$0.08            0.15$\pm$0.08   
             latentshift-max       0.33$\pm$0.07            0.21$\pm$0.09   
Effusion     grad                  0.12$\pm$0.09            0.08$\pm$0.08   
             guided                0.15$\pm$0.09            0.06$\pm$0.05   
             integrated            0.11$\pm$0.08            0.05$\pm$0.06   
             latentshift-max       0.16$\pm$0.11            0.11$\pm$0.11   
Lung Opacity grad                  0.21$\pm$0.11            0.13$\pm$0.09   
             guided                0.21$\pm$0.12            0.09$\pm$0.07   
             integrated            0.17$\pm$0.10            0.08$\pm$0.07   
             latentshift-max       0.20$\pm$0.13            0.15$\pm$0.14   
Mass         grad                  0.16$\pm$0.14                      NaN   
             guided                0.19$\pm$0.16                      NaN   
             integrated            0.13$\pm$0.13                      NaN   
             latentshift-max       0.14$\pm$0.17                      NaN   
Pneumothorax grad                  0.01$\pm$0.02            0.01$\pm$0.02   
             guided                0.03$\pm$0.05            0.02$\pm$0.03   
             integrated            0.01$\pm$0.02            0.01$\pm$0.01   
             latentshift-max       0.02$\pm$0.04            0.03$\pm$0.07   

                                                       
                                                       
model                        jfhealthcare-DenseNet121  
target       method                                    
Atelectasis  grad                       0.13$\pm$0.10  
             guided                     0.10$\pm$0.07  
             integrated                 0.10$\pm$0.09  
             latentshift-max            0.09$\pm$0.09  
Cardiomegaly grad                       0.45$\pm$0.04  
             guided                     0.31$\pm$0.05  
             integrated                 0.36$\pm$0.09  
             latentshift-max            0.35$\pm$0.09  
Effusion     grad                       0.18$\pm$0.10  
             guided                     0.14$\pm$0.07  
             integrated                 0.14$\pm$0.09  
             latentshift-max            0.16$\pm$0.10  
Lung Opacity grad                                 NaN  
             guided                               NaN  
             integrated                           NaN  
             latentshift-max                      NaN  
Mass         grad                                 NaN  
             guided                               NaN  
             integrated                           NaN  
             latentshift-max                      NaN  
Pneumothorax grad                                 NaN  
             guided                               NaN  
             integrated                           NaN  
             latentshift-max                      NaN

In [195]:
print(kk.unstack(level=1).to_latex(escape=False, ))

\begin{tabular}{lllll}
\toprule
             & {} & \multicolumn{3}{l}{iou} \\
             & {} \\
             & model & XRV-DenseNet121-all & XRV-DenseNet121-mimic_ch & jfhealthcare-DenseNet121 \\
target & method &                     &                          &                          \\
\midrule
Atelectasis & grad &       0.07$\pm$0.07 &            0.06$\pm$0.07 &            0.13$\pm$0.10 \\
             & guided &       0.09$\pm$0.08 &            0.04$\pm$0.04 &            0.10$\pm$0.07 \\
             & integrated &       0.05$\pm$0.05 &            0.04$\pm$0.05 &            0.10$\pm$0.09 \\
             & latentshift-max &       0.11$\pm$0.12 &            0.08$\pm$0.11 &            0.09$\pm$0.09 \\
Cardiomegaly & grad &       0.35$\pm$0.05 &            0.25$\pm$0.09 &            0.45$\pm$0.04 \\
             & guided &       0.28$\pm$0.06 &            0.15$\pm$0.06 &            0.31$\pm$0.05 \\
             & integrated &       0.27$\pm$0.08 &            0.15$\pm$0.08 &      

In [8]:
for_eval = [
            "Cardiomegaly",
            'Mass',
            "Atelectasis",
            "Effusion",
            "Lung Opacity",
            "Pneumothorax"
            ]

In [1]:
ds = ["nih", "pc", "rsna", "siim", "mimic_ch", "chex", "google", "openi"]

In [66]:
totals = pd.DataFrame()
for dname in ds:
    d = datasets.get_data(dname)
    xrv.datasets.relabel_dataset(for_eval, d)


    dd = pd.DataFrame(d.labels, columns=for_eval)
    g = dd.sum()
    g["Dataset"] = dname

    totals = totals.append(g, ignore_index=True)

NIH_Dataset num_samples=67310 views=['PA'] data_aug=None
{'Pneumonia', 'Consolidation', 'Hernia', 'Nodule', 'Fibrosis', 'Emphysema', 'Edema', 'Infiltration', 'Pleural_Thickening'} will be dropped
Lung Opacity doesn't exist. Adding nans instead.
PC_Dataset num_samples=91658 views=['PA'] data_aug=None
{'Hernia', 'Nodule', 'Scoliosis', 'Aortic Atheromatosis', 'Aortic Elongation', 'Hemidiaphragm Elevation', 'Granuloma', 'Bronchiectasis', 'Support Devices', 'Air Trapping', 'Fibrosis', 'Fracture', 'Pleural_Thickening', 'Pneumonia', 'Consolidation', 'Costophrenic Angle Blunting', 'Hilar Enlargement', 'Flattened Diaphragm', 'Emphysema', 'Tuberculosis', 'Edema', 'Infiltration'} will be dropped
Lung Opacity doesn't exist. Adding nans instead.
RSNA_Pneumonia_Dataset num_samples=14511 views=['PA'] data_aug=None
{'Pneumonia'} will be dropped
Cardiomegaly doesn't exist. Adding nans instead.
Mass doesn't exist. Adding nans instead.
Atelectasis doesn't exist. Adding nans instead.
Effusion doesn't exis

In [68]:
totals.set_index("Dataset")

,Atelectasis,Cardiomegaly,Effusion,Lung Opacity,Mass,Pneumothorax
Dataset,,,,,,
nih,5728.0,1563.0,6589.0,0.0,3567.0,3407.0
pc,3981.0,8420.0,3342.0,0.0,806.0,223.0
rsna,0.0,0.0,0.0,1348.0,0.0,0.0
siim,0.0,0.0,0.0,0.0,0.0,3576.0
mimic_ch,10076.0,9831.0,12064.0,13825.0,0.0,2350.0
chex,3195.0,2909.0,8078.0,9736.0,0.0,1802.0
google,0.0,0.0,0.0,221.0,0.0,46.0
openi,271.0,185.0,120.0,327.0,6.0,14.0


In [70]:
print(totals.set_index("Dataset").astype(int).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  Atelectasis &  Cardiomegaly &  Effusion &  Lung Opacity &  Mass &  Pneumothorax \\
Dataset  &              &               &           &               &       &               \\
\midrule
nih      &         5728 &          1563 &      6589 &             0 &  3567 &          3407 \\
pc       &         3981 &          8420 &      3342 &             0 &   806 &           223 \\
rsna     &            0 &             0 &         0 &          1348 &     0 &             0 \\
siim     &            0 &             0 &         0 &             0 &     0 &          3576 \\
mimic\_ch &        10076 &          9831 &     12064 &         13825 &     0 &          2350 \\
chex     &         3195 &          2909 &      8078 &          9736 &     0 &          1802 \\
google   &            0 &             0 &         0 &           221 &     0 &            46 \\
openi    &          271 &           185 &       120 &           327 &     6 &            14 \\
\bottomrule
